In [44]:
import os
print(spark.version)

2.3.1


In [45]:
os.listdir()

['.config',
 'Iris.csv',
 'spark-2.3.1-bin-hadoop2.7',
 'spark-warehouse',
 'spark-2.3.1-bin-hadoop2.7.tgz.1',
 'spark-2.3.1-bin-hadoop2.7.tgz',
 'sample_data']

In [0]:
df=spark.read.csv('Iris.csv',header=True,inferSchema=True)

In [47]:
df.show(2)

+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
+---+-------------+------------+-------------+------------+-----------+
only showing top 2 rows



In [49]:
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)



In [50]:
df.show(2)

+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
+---+-------------+------------+-------------+------------+-----------+
only showing top 2 rows



In [0]:
from pyspark.ml.feature import StringIndexer
indexer=StringIndexer(inputCol='Species',outputCol='label')
indexer=indexer.fit(df)
df=indexer.transform(df)

In [52]:
df.show(2)

+---+-------------+------------+-------------+------------+-----------+-----+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|label|
+---+-------------+------------+-------------+------------+-----------+-----+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|  0.0|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|  0.0|
+---+-------------+------------+-------------+------------+-----------+-----+
only showing top 2 rows



In [0]:
mapping=df.select('Species','label').distinct()

In [54]:
mapping.show()

+---------------+-----+
|        Species|label|
+---------------+-----+
|    Iris-setosa|  0.0|
| Iris-virginica|  2.0|
|Iris-versicolor|  1.0|
+---------------+-----+



In [55]:
df.show(2)

+---+-------------+------------+-------------+------------+-----------+-----+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|label|
+---+-------------+------------+-------------+------------+-----------+-----+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|  0.0|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|  0.0|
+---+-------------+------------+-------------+------------+-----------+-----+
only showing top 2 rows



In [0]:
from pyspark.ml.feature import VectorAssembler
assembler=VectorAssembler(inputCols=df.columns[1:-2],outputCol='features')
df_trans=assembler.transform(df)

In [60]:
df_trans.show(2)

+---+-------------+------------+-------------+------------+-----------+-----+-----------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|label|         features|
+---+-------------+------------+-------------+------------+-----------+-----+-----------------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|  0.0|[5.1,3.5,1.4,0.2]|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|  0.0|[4.9,3.0,1.4,0.2]|
+---+-------------+------------+-------------+------------+-----------+-----+-----------------+
only showing top 2 rows



In [0]:
training_data,testing_data=df_trans.randomSplit([0.8,0.2],seed=0.2)

In [76]:
training_data.show(2)

+---+-------------+------------+-------------+------------+-----------+-----+-----------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|label|         features|
+---+-------------+------------+-------------+------------+-----------+-----+-----------------+
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|  0.0|[4.9,3.0,1.4,0.2]|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|  0.0|[4.7,3.2,1.3,0.2]|
+---+-------------+------------+-------------+------------+-----------+-----+-----------------+
only showing top 2 rows



In [77]:
testing_data.show(2)

+---+-------------+------------+-------------+------------+-----------+-----+-----------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|label|         features|
+---+-------------+------------+-------------+------------+-----------+-----+-----------------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|  0.0|[5.1,3.5,1.4,0.2]|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|  0.0|[4.6,3.1,1.5,0.2]|
+---+-------------+------------+-------------+------------+-----------+-----+-----------------+
only showing top 2 rows



In [78]:
training_data.count()

120

In [79]:
testing_data.count()

30

In [0]:
from pyspark.ml.classification import DecisionTreeClassifier
model=DecisionTreeClassifier()
model=model.fit(training_data)
predictions=model.transform(testing_data)

In [91]:
predictions=predictions.withColumn('correct',F.when(F.col('label')==F.col('prediction'),'YES').otherwise('NO'))
predictions.groupBy('correct').agg(F.count('*')).show()

+-------+--------+
|correct|count(1)|
+-------+--------+
|    YES|      29|
|     NO|       1|
+-------+--------+

